In [21]:
from Optimization.Algorithms.Algorithm import Algorithm
from Numerical_Objects.Vector import Vector

import numpy as np 

In [22]:
class DolphinSearch(Algorithm):

    def __init__(self, N, T1, T2, speed, M, A, e):
        self._Dol = None
        self._k = None
        self._f_k = None
        self._l = None
        self._TS = np.full((N, N), np.inf)
        self._N = N
        self._T1 = T1
        self._T2 = T2
        self._speed = speed
        self._M = M
        self._A = A
        self._e = e

    @classmethod
    def from_dict(cls, dict_data):
        raise Exception("I don't want to do it and I'll not")

    @classmethod
    def from_json(cls, json_data):
        raise Exception('To be not-json or not to be not-json?')

    def to_dict(self):
        raise Exception('Why?')

    def to_json(self):
        raise Exception('No.')

    @property
    def current_state(self):
        raise Exception("U definitely don't need it!")

    @property
    def iterations(self):
        return [self.get_new_solution,
                # self.information_transfer,
                # self.get_optimal_solution
        ]

    def initialize(self, f, area, seed = None):
        self._Dol = []
        self._f_k = []
        for i in range(self._N):
            point = {k: np.random.uniform(low, up) for k, (low, up) in area.items()}
            point = Vector(point)
            self._Dol.append(point)
            self._f_k.append(f(point))
        self._k = [dol for dol in self._Dol]
        for f in alg._f_k:
            print(f)

    def get_new_solution(self, f, area):
        V_dol = []
        solution = []
        fitness = []
        self._l = []
        fit_l = []
        area_v = {k: (-1, 1) for k in area.keys()}

        for i in range(self._N):
            V = []
            for m in range(self._M):
                velocity = Vector({k: np.random.uniform(low, up) for k, (low, up) in area_v.items()})
                v = self._speed * (velocity * (1.0 / velocity.length))
                V.append(v)
            V_dol.append(V)
        for i in range(self._N):
            for v in V_dol[i]:
                print(i, 'V: ', v)

        for i in range(self._N):
            sol = [self._Dol[i] + v * t for v in V_dol[i] for t in range(1, self._T1 + 1)]
            print('sol: ', sol)
            fit = list(map(f, sol))
            solution.append(sol)
            fitness.append(fit)
        
        for i in range(self._N):
            idx = np.argmin(fitness[i])
            sol = solution[i]
            sol_l = sol[idx]
            self._l.append(sol_l)
            print(self._l[i])
            fit_l.append(f(sol_l))
            print(fit_l[i])

            if fit_l[i] < self._f_k[i]:
                self._k[i] = self._l[i]
                self._f_k[i] = fit_l[i]
        # return self.information_transfer

In [23]:
alg = DolphinSearch(N=2, T1=2, T2=3, speed=5, M=2, A=5, e=2)

def mul(x):
    result = 0
    for val in x.values:
        result += val ** 2 
    return result

area = {'x1': (-10, 10), 'x2': (-10, 10)}
alg.initialize(mul, area)
alg.get_new_solution(mul, area)

48.97259159074484
87.4627452733481
0 V:  (x1: 4.504844577287567) x (x2: 2.1694182018418653)
0 V:  (x1: 0.8019864079534904) x (x2: -4.93526268819177)
1 V:  (x1: 4.43203024363073) x (x2: 2.314542702034796)
1 V:  (x1: -4.629901111378755) x (x2: -1.8878600845544047)
sol:  [<Numerical_Objects.Vector.Vector object at 0x7fd051aee438>, <Numerical_Objects.Vector.Vector object at 0x7fd051aee390>, <Numerical_Objects.Vector.Vector object at 0x7fd051aee3c8>, <Numerical_Objects.Vector.Vector object at 0x7fd051aee4a8>]
sol:  [<Numerical_Objects.Vector.Vector object at 0x7fd051aee358>, <Numerical_Objects.Vector.Vector object at 0x7fd051aee2b0>, <Numerical_Objects.Vector.Vector object at 0x7fd051aee2e8>, <Numerical_Objects.Vector.Vector object at 0x7fd051aee400>]
(x1: 7.402966123332055) x (x2: -7.258979191534116)
107.49668632632732
(x1: 2.296399788107836) x (x2: -8.171895501634685)
72.05332807645891


In [24]:
for f in alg._f_k:
    print(f)

48.97259159074484
72.05332807645891
